In [10]:
'''
Escreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e criar um pandas df em que cada registro tem a coluna paper_id, que é um número sequencial.
'''

'\nEscreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e criar um pandas df em que cada registro tem a coluna paper_id, que é um número sequencial.\n'

# TODO
# Guardar Título e Journal dos papers
# Guardar Ref_Context

In [37]:
'''
prompt:
Write a function parse_tei(tei_filepath) that opens the TEI XML file and counts the number of biblStruct entries.  Also, count the number of <s></s> entries. 
Return three values: the number of s entries, the number of biblStruct entries and a list of all text inside <ref></ref> tags. 
'''

import xml.etree.ElementTree as ET

def parse_tei(tei_filepath):
    """
    Abre um arquivo TEI XML e retorna:
    - número de elementos <s> (geralmente frases)
    - número de elementos <biblStruct> (referências bibliográficas)
    - lista com o texto de todos os elementos <ref>

    Parâmetros:
        tei_filepath (str): Caminho para o arquivo TEI XML.

    Retorna:
        tuple: (int, int, list of str)
    """
    try:
        tree = ET.parse(tei_filepath)
        root = tree.getroot()

        # Detecta namespace se houver
        ns = {'tei': root.tag.split('}')[0].strip('{')} if '}' in root.tag else {}

        # Conta elementos <s>
        s_elements = root.findall('.//tei:s', ns) if ns else root.findall('.//s')
        num_s = len(s_elements)

        # Conta elementos <biblStruct>
        bibl_structs = root.findall('.//tei:biblStruct', ns) if ns else root.findall('.//biblStruct')
        num_bibl_structs = len(bibl_structs)

        # Coleta textos de <ref>
        ref_elements = root.findall('.//tei:ref', ns) if ns else root.findall('.//ref')
        ref_texts = [ref.text.strip() for ref in ref_elements if ref.text and ref.text.strip()]

        return num_s, num_bibl_structs, ref_texts

    except ET.ParseError as e:
        print(f"Erro ao analisar o XML '{tei_filepath}': {e}")
        return 0, 0, []
    except FileNotFoundError:
        print(f"Arquivo não encontrado: {tei_filepath}")
        return 0, 0, []



paper_path = "../data/teis/from-scopus/A-Historical-Intervention-in-the-Opportunity-Wars-Forgotten-Scholarship-the-DiscoveryCreation-Disruption-and-Moving-Forward-by-Looking-Backward_2023_SAGE-Publications-Ltd.pdf.grobid.tei.xml"

sentence_count, reference_count, refs = parse_tei(paper_path) 
assert sentence_count == 283
assert reference_count == 106
assert len(refs) == 222


refs

["Shane and Venkataraman's (2000)",
 '(Alvarez & Barney, 2007;',
 'Dimov, 2007;',
 'McMullen, 2015;',
 'Sarasvathy et al., 2020;',
 'Shane, 2003;',
 '2012;',
 'Suddaby et al., 2015;',
 'Wood & McKinley, 2010)',
 '(Alvarez & Porac, 2020;',
 'Arikan et al., 2020;',
 'Berglund et al., 2020;',
 'McBride & Wuebker, 2021)',
 '(Davidsson, 2015;',
 'Kitching Rouse, 2017)',
 'Foss and Klein (2020)',
 'Alvarez and Barney (2020)',
 '[and]',
 '(Alvarez & Barney, 2020, p. 306)',
 'Shane and Venkataraman (2000)',
 "Shane and Venkataraman's (2000)",
 '(Humphrey, 2005)',
 'Dutton et al., 1990;',
 'Thomas & McDaniel, 1990;',
 'Thomas et al., 1993)',
 '(Dutton & Jackson, 1987;',
 'Fredrickson, 1985)',
 '(Gartner et al., 2016)',
 '(Dutton et al., 1983;',
 'Thomas & McDaniel, 1990)',
 '(Jackson & Dutton, 1988)',
 '(Ramoglou & McMullen, 2021)',
 '(Dutton & Ottensmeyer, 1987;',
 'Ginsberg & Venkatraman, 1992)',
 '(Ginsberg & Venkataraman, 1992;',
 'Thomas & McDaniel, 1990)',
 "Shane and Venkataraman's (2000

In [35]:
'''
prompt:
Escreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e cria dois dataframes.
Um pandas df chamado "papers_df" em que cada registro tem a coluna paper_id, que é um número sequencial. 
Para cada arquivo, invoque parse_tei(tei_filepath), que retorna sentence_count, reference_count e refs.  
Adicione sentence_count e reference_count no papers_df.  
Ao outro df, chamado refs_df, adicione todos os refs. 
'''

import os
import pandas as pd


def read_tei_papers(path):
    """
    Lê todos os arquivos XML de `path` e cria dois DataFrames:
    - papers_df: com paper_id, filename, sentence_count, reference_count
    - refs_df: com paper_id e ref_text (uma linha por referência)

    Parâmetros:
        path (str): Caminho para o diretório com arquivos XML.

    Retorna:
        tuple: (papers_df, refs_df)
    """
    xml_files = [f for f in os.listdir(path) if f.lower().endswith('.xml')]

    papers_records = []
    refs_records = []

    for i, filename in enumerate(xml_files):
        paper_id = i + 1
        filepath = os.path.join(path, filename)

        sentence_count, reference_count, refs = parse_tei(filepath)

        papers_records.append({
            'paper_id': paper_id,
            'filename': filename,
            'sentence_count': sentence_count,
            'reference_count': reference_count
        })

        for ref_text in refs:
            refs_records.append({
                'paper_id': paper_id,
                'ref_text': ref_text
            })

    papers_df = pd.DataFrame(papers_records)
    refs_df = pd.DataFrame(refs_records)

    return papers_df, refs_df


In [36]:
papers_df, refs_df = read_tei_papers("../data/teis/from-scopus")

papers_df.sort_values(by='reference_count', ascending=False)

papers_df.to_csv("../data/papers.csv")
refs_df.to_csv("../data/refs.csv")

Erro ao analisar o XML '../data/teis/from-scopus\Property-is-only-another-name-for-decentralized-creation-of-knowledge_2019_Springer-New-York-LLC-barbarabbertramgskcom.pdf.grobid.tei.xml': no element found: line 1, column 0
